In [1]:
# numpy and pandas for data manipulation

from __future__ import print_function
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [11]:
df_2 = pd.read_csv("C:\\Users\\Varad Satam\\DSIE Dropbox\\Varad Satam\\Data_Mining\\feature_set_3.csv")


In [13]:
df_2=df_2.drop(['Unnamed: 0'], axis=1) #'SK_ID_CURR' add this for set 1 and 2. 3 is already without ID. 

In [14]:
df_2.head()

,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,client_bureau_balance_STATUS_5_count_min,client_bureau_balance_STATUS_C_count_mean,client_bureau_balance_STATUS_C_count_min,client_bureau_balance_STATUS_C_count_norm_max,client_bureau_balance_STATUS_X_count_mean,client_bureau_balance_STATUS_X_count_max,client_bureau_balance_STATUS_X_count_min,client_bureau_balance_STATUS_X_count_norm_mean,client_bureau_balance_STATUS_X_count_norm_max,client_bureau_balance_STATUS_X_count_norm_min
0,1.0,0.0,0.0,1.0,0.0,202500.0,406597.5,24700.5,0.018801,9461.0,...,0.0,2.875,0.0,0.8125,1.875,3.0,0.0,0.161932,0.500000,0.0
1,0.0,0.0,0.0,0.0,0.0,270000.0,1293502.5,35698.5,0.003541,16765.0,...,0.0,9.000,0.0,0.7500,1.500,4.0,0.0,0.111111,0.333333,0.0
2,0.0,1.0,1.0,1.0,0.0,67500.0,135000.0,6750.0,0.010032,19046.0,...,0.0,9.000,0.0,0.7500,1.500,4.0,0.0,0.111111,0.333333,0.0
3,0.0,0.0,0.0,1.0,0.0,135000.0,312682.5,29686.5,0.008019,19005.0,...,0.0,9.000,0.0,0.7500,1.500,4.0,0.0,0.111111,0.333333,0.0
4,0.0,0.0,0.0,1.0,0.0,121500.0,513000.0,21865.5,0.028663,19932.0,...,0.0,9.000,0.0,0.7500,1.500,4.0,0.0,0.111111,0.333333,0.0


In [15]:
def svmscores(df):
    array=np.random.randint(0,len(df),size=10000)
    ###PLit stuff into train and test
    df=df.iloc[array,:]
    X = df.drop('TARGET', axis=1)  
    y = df['TARGET']

    from sklearn import svm
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=109) # 70% training and 30% test
    
    ###Scaling cause this increases time of computation##
    from sklearn.preprocessing import MinMaxScaler
    scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
    X_train = scaling.transform(X_train)
    X_test = scaling.transform(X_test)
    
    ### prep basic model
    from sklearn.svm import SVC  
    svclassifier = SVC(kernel='rbf')
    svclassifier.fit(X_train, y_train)
    
    from sklearn.metrics import recall_score,precision_score,accuracy_score,classification_report, confusion_matrix  
    
    y_pred = svclassifier.predict(X_train)
    
    #first precision score
    unsample_prep=precision_score(y_train,y_pred)
    #First Recall Score
    unsample_recall=recall_score(y_train,y_pred)
    
    print(confusion_matrix(y_train,y_pred))
    print(accuracy_score(y_train,y_pred))
    
    #Take new balanced sets and fit model on balanced set
    from imblearn.under_sampling import RandomUnderSampler
    rus = RandomUnderSampler(random_state=0)
    
    X_undersample, Y_undersample = rus.fit_resample(X_train, y_train)


    parameters = {'kernel':('linear', 'rbf'), 'C':[0.1,0.01,0.001,0.0001]}
    svc = svm.SVC(gamma="scale")
    svm1 = GridSearchCV(svc, parameters, cv=5)
    svm1.fit(X_undersample, Y_undersample)
    
    y_pred_sample = svm1.predict(X_train)
    
    print("Accuracy Train:",accuracy_score(y_train,y_pred_sample))
    print(confusion_matrix(y_train,y_pred_sample))  
    print(classification_report(y_train,y_pred_sample))

    sample_train_precision=precision_score(y_train,y_pred_sample)
    sample_train_recall=recall_score(y_train,y_pred_sample)
    
    
    test_pred = svm1.predict(X_test)
    
    sample_test_precision = precision_score(y_test,test_pred)
    sample_test_recall = recall_score(y_test,test_pred)
    
    print("Accuracy Test:",accuracy_score(y_test,test_pred))
    print(confusion_matrix(y_test,test_pred))  
    print(classification_report(y_test,test_pred))
    
    return unsample_prep, unsample_recall, sample_train_precision, sample_train_recall, sample_test_precision, sample_test_recall

In [16]:
a,b,c,d,e,f=svmscores(df_2)

C:\Users\Varad Satam\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[[6441    0]
 [ 559    0]]
0.9201428571428572
Accuracy Train: 0.6877142857142857
[[4430 2011]
 [ 175  384]]
              precision    recall  f1-score   support

         0.0       0.96      0.69      0.80      6441
         1.0       0.16      0.69      0.26       559

   micro avg       0.69      0.69      0.69      7000
   macro avg       0.56      0.69      0.53      7000
weighted avg       0.90      0.69      0.76      7000

Accuracy Test: 0.68
[[1899  885]
 [  75  141]]
              precision    recall  f1-score   support

         0.0       0.96      0.68      0.80      2784
         1.0       0.14      0.65      0.23       216

   micro avg       0.68      0.68      0.68      3000
   macro avg       0.55      0.67      0.51      3000
weighted avg       0.90      0.68      0.76      3000



0.6296296296296297